# Question 4

In [1]:
import pandas as pd

In [2]:
df_customers = pd.read_csv('../prompts/4/customers.csv')
df_addresses = pd.read_csv('../prompts/4/addresses.csv')
df_orders = pd.read_csv('../prompts/4/orders.csv')

In [4]:
df_addresses

,addressID,customerID,dt_eff,dt_ends,zip_code
0,1,1,2000-01-01,2000-05-11,69431
1,2,1,2000-05-11,2015-11-04,91336
2,3,1,2015-11-04,9999-12-31,67834
3,4,2,2000-01-01,2009-05-14,21803
4,5,2,2009-05-14,2016-08-05,99238
...,...,...,...,...,...
244,245,98,2000-01-01,2001-11-23,71251
245,246,98,2001-11-23,2012-04-09,34006
246,247,98,2012-04-09,9999-12-31,83960
247,248,99,2000-01-01,9999-12-31,64332


In [6]:
df_orders

,orderID,datetime,customerID
0,1,2008-04-12,14
1,2,2004-03-14,94
2,3,2010-06-12,23
3,4,2005-05-03,96
4,5,2019-10-19,71
...,...,...,...
995,996,2005-08-30,43
996,997,2019-08-21,70
997,998,2018-02-04,50
998,999,2004-06-29,56


In [7]:
import datetime

def make_dt_object( string ):
    return datetime.datetime.strptime( string, '%Y-%m-%d' )


In [8]:
df_orders['dt_obj'] = df_orders['datetime'].apply( lambda x: make_dt_object(x) )

In [9]:
df_addresses['dt_eff_obj'] = df_addresses['dt_eff'].apply( lambda x: make_dt_object(x))
df_addresses['dt_ends_obj'] = df_addresses['dt_ends'].apply( lambda x: make_dt_object(x))


In [10]:
df_addresses

,addressID,customerID,dt_eff,dt_ends,zip_code,dt_eff_obj,dt_ends_obj
0,1,1,2000-01-01,2000-05-11,69431,2000-01-01,2000-05-11 00:00:00
1,2,1,2000-05-11,2015-11-04,91336,2000-05-11,2015-11-04 00:00:00
2,3,1,2015-11-04,9999-12-31,67834,2015-11-04,9999-12-31 00:00:00
3,4,2,2000-01-01,2009-05-14,21803,2000-01-01,2009-05-14 00:00:00
4,5,2,2009-05-14,2016-08-05,99238,2009-05-14,2016-08-05 00:00:00
...,...,...,...,...,...,...,...
244,245,98,2000-01-01,2001-11-23,71251,2000-01-01,2001-11-23 00:00:00
245,246,98,2001-11-23,2012-04-09,34006,2001-11-23,2012-04-09 00:00:00
246,247,98,2012-04-09,9999-12-31,83960,2012-04-09,9999-12-31 00:00:00
247,248,99,2000-01-01,9999-12-31,64332,2000-01-01,9999-12-31 00:00:00


## get 2010 orders

In [11]:
df_2010_orders = df_orders[ df_orders['dt_obj'].apply( lambda x: x.year==2010 )  ] 

In [12]:
df_2010_orders.head(10)

,orderID,datetime,customerID,dt_obj
2,3,2010-06-12,23,2010-06-12
5,6,2010-03-29,85,2010-03-29
14,15,2010-11-24,4,2010-11-24
15,16,2010-11-05,30,2010-11-05
46,47,2010-05-03,18,2010-05-03
70,71,2010-02-21,69,2010-02-21
96,97,2010-08-25,44,2010-08-25
101,102,2010-04-17,91,2010-04-17
117,118,2010-12-19,49,2010-12-19
118,119,2010-09-29,48,2010-09-29


## match on current address

In [13]:
def get_current_zip( customerID, datetime ):
    
    df_customer_address = df_addresses[ df_addresses['customerID'] == customerID ]    
    df_active = df_customer_address[ df_customer_address['dt_eff_obj'] <= datetime ][ df_customer_address['dt_ends_obj'] > datetime ]
    
    return df_active['zip_code'].values[0]


In [14]:
get_current_zip( 1, datetime.datetime(2018,1,1) )  

67834

In [15]:
df_2010_orders['zip_code'] = df_2010_orders[ ['customerID','dt_obj'] ].apply( lambda x: get_current_zip(x['customerID'], x['dt_obj']), axis=1 )

/var/folders/dj/_mwbb77n38zgmcn3yx6m2ff00000gn/T/ipykernel_43462/1954968820.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_active = df_customer_address[ df_customer_address['dt_eff_obj'] <= datetime ][ df_customer_address['dt_ends_obj'] > datetime ]
/var/folders/dj/_mwbb77n38zgmcn3yx6m2ff00000gn/T/ipykernel_43462/2338961012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2010_orders['zip_code'] = df_2010_orders[ ['customerID','dt_obj'] ].apply( lambda x: get_current_zip(x['customerID'], x['dt_obj']), axis=1 )


In [16]:
df_2010_orders

,orderID,datetime,customerID,dt_obj,zip_code
2,3,2010-06-12,23,2010-06-12,60770
5,6,2010-03-29,85,2010-03-29,22981
14,15,2010-11-24,4,2010-11-24,96093
15,16,2010-11-05,30,2010-11-05,85673
46,47,2010-05-03,18,2010-05-03,82950
70,71,2010-02-21,69,2010-02-21,66090
96,97,2010-08-25,44,2010-08-25,49072
101,102,2010-04-17,91,2010-04-17,69034
117,118,2010-12-19,49,2010-12-19,67389
118,119,2010-09-29,48,2010-09-29,70126


In [18]:
round(df_2010_orders['zip_code'].mean())

55489